In [6]:
# Import Library and training the data
import random
import spacy
from spacy.util import minibatch
from spacy.training.example import Example

train_data = [
    ("What is the price of 10 bananas?", {"entities": [(21, 23, "QUANTITY"), (24, 31, "PRODUCT")]}),
    ("What is the price of 5 laptops?", {"entities": [(21, 22, "QUANTITY"), (23, 30, "PRODUCT")]}),
    ("How much are 7 bottles?", {"entities": [(13, 14, "QUANTITY"), (15, 22, "PRODUCT")]}),
    ("Could I buy 17 phones from you?", {"entities": [(12, 14, "QUANTITY"), (15, 21, "PRODUCT")]}),
    ("I am interested in acquiring 10 books.", {"entities": [(31, 33, "QUANTITY"), (34, 39, "PRODUCT")]}),
    ("Can you get me 12 apples?", {"entities": [(16, 18, "QUANTITY"), (19, 25, "PRODUCT")]}),
    ("Please check the price of 3 pens.", {"entities": [(26, 27, "QUANTITY"), (28, 32, "PRODUCT")]}),
    ("What do 5 chairs cost?", {"entities": [(8, 9, "QUANTITY"), (10, 16, "PRODUCT")]}),
    ("I need 8 laptops for the office.", {"entities": [(7, 8, "QUANTITY"), (9, 16, "PRODUCT")]}),
    ("How much for 4 desks?", {"entities": [(13, 14, "QUANTITY"), (15, 20, "PRODUCT")]}),
    ("We require 20 notebooks.", {"entities": [(11, 13, "QUANTITY"), (14, 23, "PRODUCT")]}),
    ("I want to order 6 cameras.", {"entities": [(17, 18, "QUANTITY"), (19, 26, "PRODUCT")]}),
    ("Can you provide the price of 2 watches?", {"entities": [(28, 29, "QUANTITY"), (30, 37, "PRODUCT")]}),
    ("Tell me the cost of 3 bottles.", {"entities": [(20, 21, "QUANTITY"), (22, 29, "PRODUCT")]}),
    ("Is it possible to buy 7 microphones?", {"entities": [(24, 25, "QUANTITY"), (26, 37, "PRODUCT")]}),
    ("I would like 10 keyboards.", {"entities": [(14, 16, "QUANTITY"), (17, 26, "PRODUCT")]}),
    ("Give me 15 calculators.", {"entities": [(8, 10, "QUANTITY"), (11, 22, "PRODUCT")]}),
    ("Could I get 9 mouse pads?", {"entities": [(12, 13, "QUANTITY"), (14, 24, "PRODUCT")]}),
    ("Can you tell me about 6 monitors?", {"entities": [(20, 21, "QUANTITY"), (22, 30, "PRODUCT")]}),
    ("I'm looking for 11 hard drives.", {"entities": [(16, 18, "QUANTITY"), (19, 30, "PRODUCT")]}),
    ("Do you have 5 power banks?", {"entities": [(12, 13, "QUANTITY"), (14, 25, "PRODUCT")]}),
    ("How about 14 USB drives?", {"entities": [(10, 12, "QUANTITY"), (13, 23, "PRODUCT")]}),
    ("We need 18 projectors for the event.", {"entities": [(8, 10, "QUANTITY"), (11, 21, "PRODUCT")]}),
    ("Could you send 4 printers to the office?", {"entities": [(13, 14, "QUANTITY"), (15, 23, "PRODUCT")]}),
    ("Are 3 scanners available?", {"entities": [(4, 5, "QUANTITY"), (6, 14, "PRODUCT")]}),
]


In [16]:
#load the model(fine tune)
!python -m spacy download en_core_web_md
nlp=spacy.load('en_core_web_md')

if 'ner' not in nlp.pipe_names:
    ner=nlp.add_pipe('ner')  #add
else:
    ner=nlp.get_pipe('ner')  #get existing

  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_md-3.8.0/en_core_web_md-3.8.0-py3-none-any.whl (33.5 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [8]:
#add entities
for _,annotations in train_data:
  for ent in annotations.get('entities'):
    if ent[2] not in ner.labels:
      ner.add_label(ent[2])

In [24]:
from spacy.lang.en import English
from spacy.lookups import Lookups

nlp = English()
lookups = Lookups()
nlp.vocab.lookups = lookups


In [30]:
other_pipes=[pipe for pipe in nlp.pipe_names if pipe!='ner']
!pip install spacy-lookups-data # Explicitly install spacy-lookups-data


if 'ner' not in nlp.pipe_names:
    # Add the NER pipe to the current blank nlp model
    ner_pipe = nlp.add_pipe('ner', last=True)
    # Re-add labels to this newly added NER component, as the previous `ner` variable is now stale.
    for _, annotations in train_data:
        for ent in annotations.get('entities'):
            if ent[2] not in ner_pipe.labels:
                ner_pipe.add_label(ent[2])
else:

    ner_pipe = nlp.get_pipe('ner')

with nlp.disable_pipes(*other_pipes):
  optimizer=nlp.begin_training()
  epochs=50
  for epoch in range(epochs):
    random.shuffle(train_data)
    losses={}
    batches=minibatch(train_data,size=2)
    for batch in batches:
      examples=[]
      for text,annotations in batch:
        doc=nlp.make_doc(text)

        example=Example.from_dict(doc,annotations)
        examples.append(example)
      nlp.update(examples,drop=0.5,losses=losses)
    print(f'Epoch{epoch+1},Losses:{losses}')
nlp.to_disk('customer_ner_model')
training_nlp=spacy.load('customer_ner_model')

/usr/local/lib/python3.12/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "I would like 10 keyboards." with entities "[(14, 16, 'QUANTITY'), (17, 26, 'PRODUCT')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Can you get me 12 apples?" with entities "[(16, 18, 'QUANTITY'), (19, 25, 'PRODUCT')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Could you send 4 printers to the office?" with entitie

Epoch1,Losses:{'ner': np.float32(117.96057)}
Epoch2,Losses:{'ner': np.float32(63.2355)}
Epoch3,Losses:{'ner': np.float32(56.192776)}
Epoch4,Losses:{'ner': np.float32(49.720882)}
Epoch5,Losses:{'ner': np.float32(43.973125)}
Epoch6,Losses:{'ner': np.float32(36.435246)}
Epoch7,Losses:{'ner': np.float32(25.605124)}
Epoch8,Losses:{'ner': np.float32(12.120318)}
Epoch9,Losses:{'ner': np.float32(15.016664)}
Epoch10,Losses:{'ner': np.float32(11.720258)}
Epoch11,Losses:{'ner': np.float32(14.251397)}
Epoch12,Losses:{'ner': np.float32(17.363907)}
Epoch13,Losses:{'ner': np.float32(6.0636773)}
Epoch14,Losses:{'ner': np.float32(8.894908)}
Epoch15,Losses:{'ner': np.float32(16.121908)}
Epoch16,Losses:{'ner': np.float32(12.591825)}
Epoch17,Losses:{'ner': np.float32(6.5414534)}
Epoch18,Losses:{'ner': np.float32(7.2453547)}
Epoch19,Losses:{'ner': np.float32(14.066449)}
Epoch20,Losses:{'ner': np.float32(9.389959)}
Epoch21,Losses:{'ner': np.float32(2.991284)}
Epoch22,Losses:{'ner': np.float32(5.3794317)}
Ep

In [41]:
#evaluate
test_texts=[
    "How much for 3 oranges?",
    "I want 15 chairs for the conference",
    "Can you give me the price for 6 desks?"
    ]

In [42]:
for text in test_texts:
  doc=training_nlp(text)
  print(f'Text:{text}')
  print(f'Entities',[(ent.text,ent.label_) for ent in doc.ents])
  print()

Text:How much for 3 oranges?
Entities [('3', 'QUANTITY'), ('oranges', 'PRODUCT')]

Text:I want 15 chairs for the conference
Entities [('15', 'QUANTITY'), ('chairs', 'PRODUCT')]

Text:Can you give me the price for 6 desks?
Entities [('6', 'QUANTITY'), ('desks', 'PRODUCT')]

